In [49]:
import cv2 as cv

In [50]:
image_paths = ["assets/image 1.jpg", "assets/image 2.jpg"]
min_pixels = 2
border = 5
border_final = 1
expand_size = 1.5
width_min = 0
height_min = 0
images_saved_size = 32

digits = ['#', '$', '%', '&', '1', '2', '3', '4', '5']

In [51]:
class Matrix:
    def __init__(self, col, row, img):
        self.x = col
        self.y = row
        self.img = img

        self.islands = []

    def dfs(self, i, j, visited):
        visited[i][j] = True
        self.islands[-1].append((i, j))

        for k in range(-1, 2):
            for l in range(-1, 2):
                if (k or l) and self.isSafe(i + k, j + l, visited):
                    self.dfs(i + k, j + l, visited)

    def find(self, i, j):
        return self.img[i, j] == 0

    def isSafe(self, i, j, visited) -> bool:
        row = 0 <= i < self.x
        col = 0 <= j < self.y
        return row and col and not visited[i][j] and self.find(i, j)

    def countIslands(self) -> int:
        visited = [[False for j in range(self.y)] for i in range(self.x)]

        count = 0

        for i in range(self.x):
            for j in range(self.y):
                if not visited[i][j] and self.find(i, j):
                    self.islands.append([])
                    self.dfs(i, j, visited)
                    count += 1

        return count

    def getIslands(self):
        self.islands = []
        self.countIslands()
        return list(filter(lambda item: len(item) > min_pixels, self.islands))

    def getIsland(self, i, j):
        for _island in list(filter(lambda item: len(item) > min_pixels, self.islands)):
            if (i, j) in _island:
                return _island
        raise Exception("island not found")


In [52]:
def normalize(images):
    # x_min, x_max, y_min, y_max
    for index, i in enumerate(images):
        for j in images[index:]:
            if i != j and i[2] < j[3] and i[3] > j[2] and i[0] < j[1] and i[1] > j[0]:
                _x_min = i[0] if i[0] < j[0] else j[0]
                _x_max = i[1] if i[1] > j[1] else j[1]
                _y_min = i[2] if i[2] < j[2] else j[2]
                _y_max = i[3] if i[3] > j[3] else j[3]

                _width = _x_max - _x_min
                _height = _y_max - _y_min

                if max_width * expand_size > _width and max_height * expand_size > _height:
                    crop_images.remove(j)
                    crop_images[index] = (_x_min, _x_max, _y_min, _y_max)
                    return True
    return False

In [53]:
import os
import shutil

images_path = "images/"
if os.path.exists(images_path):
    shutil.rmtree(images_path)
os.mkdir(images_path)

images_path_src = f"{images_path}/src/"
if os.path.exists(images_path_src):
    shutil.rmtree(images_path_src)
os.mkdir(images_path_src)
for digit in digits:
    os.mkdir(f"{images_path_src}{digit}/")

In [54]:
umbral = [140, 130]

for index_, image_path in enumerate(image_paths):
    print("\nindex_", index_)
    print("image_path", image_path)

    img_color = cv.imread(image_path)
    img_src = cv.imread(image_path, 0)

    img_gray = cv.cvtColor(img_color, cv.COLOR_BGR2GRAY)
    (thresh, img_bw) = cv.threshold(img_gray, umbral[index_], 255, cv.THRESH_BINARY)

    x, y = img_bw.shape[:2]
    print(f"image size: x:{x}, y:{y}")

    matrix = Matrix(x, y, img_bw)
    islands = matrix.getIslands()
    print(f"islands: {len(islands)}")

    crop_images = []
    max_width = 0
    max_height = 0

    for island in islands:
        x_max = max([t[0] for t in island]) + border
        x_min = min([t[0] for t in island]) - border
        y_max = max([t[1] for t in island]) + border
        y_min = min([t[1] for t in island]) - border

        x_max = x_max if x > x_max else x - 1
        x_min = x_min if x_min > 0 else 0
        y_max = y_max if y > y_max else y - 1
        y_min = y_min if y_min > 0 else 0

        width = x_max - x_min
        height = y_max - y_min

        if width > width_min and height > height_min:
            max_width = max_width if max_width > width else width
            max_height = max_height if max_height > height else height

            crop_images.append((x_min, x_max, y_min, y_max))

    w = True
    while w:
        temp = crop_images
        w = normalize(temp)

    for x_min, x_max, y_min, y_max in crop_images:
        x_min += border
        y_min += border
        x_max -= border
        y_max -= border

        x_min = x_min if x_min > 0 else 0
        y_min = y_min if y_min > 0 else 0
        x_max = x_max if x > x_max else x - 1
        y_max = y_max if y > y_max else y - 1

        img_color = cv.rectangle(img_color, (y_min, x_min), (y_max, x_max), (0, 0, 255), 1)
        img_color = cv.putText(img_color, str(y_min), (y_min, x_max), cv.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 255), 1, 2)

        crop_imag_temp = img_src[x_min - border_final:x_max + border_final, y_min - border_final:y_max + border_final]
        crop_imag_temp = cv.copyMakeBorder(crop_imag_temp, top=border_final, bottom=border_final, left=border_final,
                                           right=border_final, borderType=cv.BORDER_CONSTANT, value=255)

        temp_x, temp_y = crop_imag_temp.shape[:2]

        temp_border = abs(temp_x - temp_y)
        if temp_x > temp_y:
            top = 0
            bottom = 0
            left = temp_border // 2
            right = temp_border - temp_border // 2

        else:
            top = temp_border // 2
            bottom = temp_border - temp_border // 2
            left = 0
            right = 0

        square_img = cv.copyMakeBorder(crop_imag_temp, top=top, bottom=bottom, left=left, right=right,
                                       borderType=cv.BORDER_CONSTANT, value=255)

        (_, img_bw_) = cv.threshold(square_img, 140, 255, cv.THRESH_BINARY)

        digit = ""
        if index_ == 0:
            if y_min < 35:
                digit = "1"
            elif y_min < 70:
                digit = "2"
            elif y_min < 105:
                digit = "3"
            elif y_min < 140:
                digit = "4"
            elif y_min < 175:
                digit = "5"
            elif y_min < 210:
                digit = "#"
            elif y_min < 245:
                digit = "$"
            elif y_min < 280:
                digit = "%"
            else:
                digit = "&"
        elif index_ == 1:
            if y_min < 50:
                digit = "1"
            elif y_min < 100:
                digit = "2"
            elif y_min < 150:
                digit = "3"
            elif y_min < 200:
                digit = "4"
            elif y_min < 260:
                digit = "5"
            elif y_min < 310:
                digit = "#"
            elif y_min < 400:
                digit = "$"
            elif y_min < 450:
                digit = "%"
            else:
                digit = "&"

        img_resize = cv.resize(img_bw_, (32, 32))
        img_final = cv.bitwise_not(img_resize)

        cv.imwrite(
            f"{images_path_src}/{digit}/{str(x_min).rjust(4, '0')}{str(x_max).rjust(4, '0')}{str(y_min).rjust(4, '0')}{str(y_max).rjust(4, '0')}.jpg",
            img_final)
        print(
            f"{images_path_src}/{digit}/{str(x_min).rjust(4, '0')}{str(x_max).rjust(4, '0')}{str(y_min).rjust(4, '0')}{str(y_max).rjust(4, '0')}.jpg",
            end="\r")

    cv.imwrite(f"detail {index_}.jpg", img_color)
    print(f"{len(crop_images)} images saved")



index_ 0
image_path assets/image 1.jpg
image size: x:478, y:327
islands: 144
116 images saved455046800450055.jpg

index_ 1
image_path assets/image 2.jpg
image size: x:768, y:500
islands: 155
126 images saved743076100100017.jpg


In [66]:
import shutil, os

images_path_rot = f"{images_path}/rot/"
if os.path.exists(images_path_rot):
    shutil.rmtree(images_path_rot)
os.mkdir(images_path_rot)
for digit in digits:
    os.mkdir(f"{images_path_rot}{digit}/")

In [67]:
import os

rot_images = []

for root, sub_dirs, files in os.walk(images_path_src):
    for file in files:
        sub_dir = "".join(sub_dirs)
        path = f"{root}/{sub_dir}/{file}"

        img = cv.imread(path, 0)
        img = cv.resize(img, (32, 32))
        h, w = img.shape[:2]

        for i in range(-30, 31, 3):
            angle = i
            path = f"{images_path_rot}/{root.split('/')[-1]}/{str(angle).rjust(3, '0')}{file}"

            mw = cv.getRotationMatrix2D((h // 2, w // 2), angle, 1)
            img2 = cv.warpAffine(img, mw, (h, w), borderMode=cv.BORDER_CONSTANT, borderValue=(0, 0, 0))
            rot_images.append([path, img2])

for path, img in rot_images:
    cv.imwrite(path, img)
    print(f"{path}", end="\r")

print(f"\n{len(rot_images)} images saved")

images//rot//5/0303580.png02450256.jpg
5397 images saved


In [68]:
import shutil, os

images_path_per = f"{images_path}/per/"
if os.path.exists(images_path_per):
    shutil.rmtree(images_path_per)
os.mkdir(images_path_per)
for digit in digits:
    os.mkdir(f"{images_path_per}{digit}/")


In [69]:
def get_per(h, w, l1, l2, l3, l4):
    return [[0 - (w / 2) * (l1 - 1), 0 - (h / 2) * (l4 - 1)],
            [w + (w / 2) * (l1 - 1), 0 - (h / 2) * (l2 - 1)],
            [w + (w / 2) * (l3 - 1), h + (h / 2) * (l2 - 1)],
            [0 - (w / 2) * (l3 - 1), h + (h / 2) * (l4 - 1)]]


import numpy as np

per_images = []

for root, sub_dirs, files in os.walk(images_path_rot):
    for file in files:
        sub_dir = "".join(sub_dirs)
        path = f"{root}/{sub_dir}/{file}"

        img = cv.imread(path, 0)
        x, y = img.shape[:2]

        _input = np.float32([[0, 0], [y, 0], [y, x], [0, x]])
        _outputs = [
            np.float32(get_per(x, y, 1, 1, 1, 1)),
            np.float32(get_per(x, y, 1.1, 1, 1, 1)),
            np.float32(get_per(x, y, 1, 1.1, 1, 1)),
            np.float32(get_per(x, y, 1, 1, 1.1, 1)),
            np.float32(get_per(x, y, 1, 1, 1, 2)),
            np.float32(get_per(x, y, 1.1, 1.1, 1, 1)),
            np.float32(get_per(x, y, 1, 1.1, 1.1, 1)),
            np.float32(get_per(x, y, 1, 1, 1.1, 2)),
            np.float32(get_per(x, y, 1.1, 1, 1, 2)),
        ]

        for i, _output in enumerate(_outputs):
            transform = cv.getPerspectiveTransform(_input, _output)
            path = f"{images_path_per}/{root.split('/')[-1]}/{str(i).rjust(2, '0')}{file}"

            # (thresh, img) = cv.threshold(img, 105, 255, cv.THRESH_BINARY)
            perspective = cv.warpPerspective(img, transform, (x, y), cv.INTER_LINEAR, borderMode=cv.BORDER_CONSTANT,
                                             borderValue=(0, 0, 0))
            per_images.append([path, perspective])

for path, img in per_images:
    cv.imwrite(path, img)
    print(f"{path}", end="\r")

print(f"\n{len(per_images)} images saved")
print(f"\n{len(per_images) // len(digits)} images saved per digit")


mages//per//2/00-1559896.pngimages//per//2/06-240007002300480064.jpgimages//per//2/03-300083010200450065.jpgimages//per//2/010-60114013900490062.jpgimages//per//2/040000114013900490062.jpgimages//per//2/020060164018000480059.jpgimages//per//2/010120083010200450065.jpgimages//per//2/050180083010200450065.jpgimages//per//2/0102159896.pngimages//per//2/0202759757.pngimages//per//3/01-150008002701290143.jpgimages//per//3/02-210115013900820096.jpgimages//per//3/02-270121013901220138.jpgimages//per//3/030-30177019401220139.jpgimages//per//3/000-90064008101270138.jpgimages//per//3/030030064008101270138.jpgimages//per//3/010090048006500810092.jpgimages//per//3/050150083010400830100.jpgimages//per//3/070182834.pngimages//per//3/070242961.pngimages//per//3/080302740.pngimages//per//4/03-150514053601800196.jpgimages//per//4/01-210405042301820194.jpgimages//per//4/08-270405042301820194.jpgimages//per//4/070-30301032401180130.jpgimages//per//4/030-90288031101830197.jpgimages//per//4/060030165017901